In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py

import kalepy as kale

import holodeck as holo
from holodeck import cosmo, utils
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

In [ ]:
import holodeck.sam
reload(holodeck.sam)
sam = holodeck.sam.BP_Semi_Analytic()
# out = sam.dnbh_dz()
# print(f"{out.shape=}")
freqs = np.logspace(-1, 1, 3) / YR
gwb = sam.gwb_sa(freqs)
# print(f"{gwb=}")

# gwb.shape
hc = np.sqrt(np.sum(gwb, axis=(1, 2, 3)))
hc.shape, hc

In [ ]:
plt.plot(freqs, hc)
plt.gca().set(xscale='log', yscale='log')
plt.show()

In [ ]:

























































\\\\\\\'????????????????????????'

In [ ]:
UKKKKKKKKKKKKKKK∆π¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿ˆH